In [1]:
# Importing all necessary libraries

In [ ]:
# Egemen.kz

In [73]:
import requests
from bs4 import BeautifulSoup as BS
import re as re
import os
import math
from datetime import datetime
import csv
import json

In [66]:
# Function for extracting the URL for articles in Egemen.kz

In [67]:
def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('div', class_='clearfix news-t flexBlock')

    # Extract URLs from <a> tags inside those divs
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]

    return full_urls

In [68]:
# Created the function specifically for the parsing Egemen.kz web-site content and saved as a text file
# Only works for one URL 

In [69]:
def article_contents(article_url, storage_dir):
    
    try:
        # Загрузка HTML страницы
        response = requests.get(article_url)
        
        # Check for 404 errors or other HTTP status codes
        if response.status_code != 200:
            raise requests.HTTPError(f"HTTP Error: {response.status_code} for {article_url}")
        
        html_content = response.text
        
        # Парсинг HTML
        soup = BS(html_content, "html.parser")
        
        # Title extract
        try:
            title = soup.find('h1')
            title = title.get_text(strip=True)
        except Exception: 
            title = "Title not found"
        
        # Author extract
        try:
            author = soup.find('div', class_='name-auth').text.strip()
        except Exception: 
            author = "Author not found"
        
        # Date extract
        try:
            date_tag = soup.find('meta', itemprop="datePublished")
            if date_tag and date_tag.has_attr('content'):
                date_published = date_tag['content']
                date_published_dt = datetime.strptime(date_published, "%Y-%m-%d %H:%M:%S")
            else:
                date_published = "Date not found"
        except Exception: 
            date_published = "Date not found"
        
        # Content extract
        try:
            article_body = soup.find("div", itemprop="articleBody")
            article_text = article_body.get_text(separator="\n", strip=True) if article_body else "Content not found"
        except Exception: 
            article_text = "Content not found"
        
        # Сохранение текста в файл
        valid_title = re.sub(r'[\\/:"*?<>|]+', '', title)  # Remove invalid characters
        filename = os.path.join(storage_dir, f"{valid_title}.txt")
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(f"{title}\n\n")
            file.write(article_text)
        
        print("Статья успешно сохранена!")
        
        return {
        "Title": title,
        "Date Published": date_published,
        "Author": author,
        "URL": article_url,
        "Source": article_url.split('/')[2]
    }

    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [70]:
def extract_article_details(article_url):
    """Extract title, date published, author, and URL."""
    response = requests.get(article_url)
    html_content = response.text
    soup = BS(html_content, "html.parser")
    
    # Extract title
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else "No Title"
    
    # Extract date published
    date_tag = soup.find('meta', itemprop="datePublished")
    date_published = date_tag['content'] if date_tag and date_tag.has_attr('content') else "No Date"
    date_published_dt = None
    if date_published != "No Date":
        date_published_dt = datetime.strptime(date_published, "%Y-%m-%d %H:%M:%S")
        date_published = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
    
    # Extract author
    author_tag = soup.find('div', class_='name-auth')
    author = author_tag.get_text(strip=True) if author_tag else "No Author"
    
    return {
        "Title": title,
        "Date Published": date_published,
        "Author": author,
        "URL": article_url
    }


In [71]:
def main():
    
    # Key word input
    key_word_input = input("Введите ключевое слово: \n")
    
    # Creating url with key-word
    base_url_search = "https://egemen.kz"                         #base url can be changed by other URL
    search_url = base_url_search + "/search?q=" 
    words = key_word_input.split()
    string = "+".join(words)
    key_word_url = search_url + string

    print(f"Ссылка по вашему запросу: \n{key_word_url}")
    
    # Creating directory with the name of the input 
    storage_dir = os.path.join(os.getcwd(), key_word_input)
    os.makedirs(storage_dir, exist_ok=True)

    response = requests.get(key_word_url) 
    html_content = response.text
    
    # Parsing HTML
    soup = BS(html_content, "html.parser")
    
    # Finding articles number, included in web-site
    article_founded = soup.find('small').text

    print(article_founded)
    
    # Conversion to int from list
    num_article = re.findall(r'\d+', article_founded)
    num = int(num_article[0]) 
    
    # Each web-page in site only contains 5 articels, it can be also changed 
    articles_per_page = 5
    pages = math.ceil(num / articles_per_page) 

    print(f"Чило страниц:{pages}\n")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):
        full_url = key_word_url + "&page=" + str(page)
        print(f"Processing page: {page} {full_url}\n")
    
        # Call the function and get the extracted URLs
        article_urls = extract_urls(base_url_search, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to the text")
    for url in urls_list:
        print(url)
        article_contents(url,storage_dir)
        
    # CSV file creation
    csv_file = os.path.join(storage_dir, f"{key_word_input}.csv")
    
    # CSV headers
    headers = ["Title", "Date Published", "Author", "URL", "Source", "Status"]
    

        
    # Writing to CSV
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
    
        for url in urls_list:
            print(f"Processing article: {url}")
            article_details = extract_article_details(url)
            writer.writerow(article_details)
    print("CSV файл успешно сохранен !")

In [72]:
if __name__ == "__main__":
    main()

Введите ключевое слово: 
Донор
Ссылка по вашему запросу: 
https://egemen.kz/search?q=Донор
73 материал табылды
Чило страниц:15

Processing page: 1 https://egemen.kz/search?q=Донор&page=1

https://egemen.kz/article/377917-mayittik-donorlyqty-damytugha-ne-kedergi

https://egemen.kz/article/375100-qaytys-bolghan-donordynh-aghza-musheleri-4-adamnynh-omirin-saqtap-qaldy

https://egemen.kz/article/373373-suyek-kemigi-donory

https://egemen.kz/article/370779-bes-adamgha-omir-syylaghan-donor

https://egemen.kz/article/367937-donor-buyrekpen-dgurip-dombyra-unin-kokke-samghatty

Processing page: 2 https://egemen.kz/search?q=Донор&page=2

https://egemen.kz/article/367714-donor-oblys-qatary-qaytse-artady

https://egemen.kz/article/357191-mayit-donorlyghyn-qalpyna-keltiru-qadget-eks-ministr-transplantatsiyadaghy-tuytk

https://egemen.kz/article/356281-eki-donor-toghyz-adamnynh-omirin-saqtady-ministrlik-donorlyqqa-kelisken-azamatta

https://egemen.kz/article/356279-aghza-mushelerimdi-donor-retinde-b

Статья успешно сохранена!
https://egemen.kz/article/375100-qaytys-bolghan-donordynh-aghza-musheleri-4-adamnynh-omirin-saqtap-qaldy
Статья успешно сохранена!
https://egemen.kz/article/373373-suyek-kemigi-donory
Статья успешно сохранена!
https://egemen.kz/article/370779-bes-adamgha-omir-syylaghan-donor
Статья успешно сохранена!
https://egemen.kz/article/367937-donor-buyrekpen-dgurip-dombyra-unin-kokke-samghatty
Статья успешно сохранена!
https://egemen.kz/article/367714-donor-oblys-qatary-qaytse-artady
Статья успешно сохранена!
https://egemen.kz/article/357191-mayit-donorlyghyn-qalpyna-keltiru-qadget-eks-ministr-transplantatsiyadaghy-tuytk
Статья успешно сохранена!
https://egemen.kz/article/356281-eki-donor-toghyz-adamnynh-omirin-saqtady-ministrlik-donorlyqqa-kelisken-azamatta
Статья успешно сохранена!
https://egemen.kz/article/356279-aghza-mushelerimdi-donor-retinde-beruge-kelisemin-densaulyq-saqtau-ministri
Статья успешно сохранена!
https://egemen.kz/article/351147-shymkent-donor-shaqar

Processing article: https://egemen.kz/article/356281-eki-donor-toghyz-adamnynh-omirin-saqtady-ministrlik-donorlyqqa-kelisken-azamatta
Processing article: https://egemen.kz/article/356279-aghza-mushelerimdi-donor-retinde-beruge-kelisemin-densaulyq-saqtau-ministri
Processing article: https://egemen.kz/article/351147-shymkent-donor-shaqargha-aynala-ma
Processing article: https://egemen.kz/article/350387-qazaqstanda-europalyq-aghza-donorynynh-kunine-oray-donor-kuni-atap-otiledi
Processing article: https://egemen.kz/article/346275-bir-donor-tort-adamdy-qutqaryp-qaldy
Processing article: https://egemen.kz/article/345996-bir-donor-4-adamnynh-omirin-saqtap-qaldy
Processing article: https://egemen.kz/article/333286-transplantatsiyada-donor-tapshy
Processing article: https://egemen.kz/article/332119-kutu-paraghy-–-eki-dgylda-donorlyq-aghzagha-zaru-558-patsient-koz-dgumdy
Processing article: https://egemen.kz/article/327890-donor-bolu-–-dgauapty-is
Processing article: https://egemen.kz/article/32

In [ ]:
###### Tengri-news

In [ ]:
## https://tengrinews.kz 

In [115]:
def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('div', class_='content_main_item')

    # Extract URLs from <a> tags inside those divs
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]

    return full_urls

In [116]:
def save_to_csv(data_list, csv_file, headers):
    """Saves a list of article details to a CSV file."""
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data_list)
    print(f"CSV файл успешно сохранен в {csv_file}!")


In [117]:
def extract_article_details(article_url):
    """Extract title, date published, author, and URL."""
    response = requests.get(article_url)
    html_content = response.text
    soup = BS(html_content, "html.parser")
    
    # Extract title
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else "No Title"
    
    # Extract date published
    json_ld_tags = soup.find_all('script', type="application/ld+json")
    date_published = "No Date"

    # Iterate through the tags to find the one with 'datePublished'
    for tag in json_ld_tags:
        try:
            # Parse the JSON content
            json_ld = json.loads(tag.string)
        
            # Check if it's a dictionary and has 'datePublished'
            if isinstance(json_ld, dict) and 'datePublished' in json_ld:
                date_published = json_ld['datePublished']
                break
            elif isinstance(json_ld, list):  # JSON-LD can also be a list of objects
                for item in json_ld:
                    if '@type' in item and item['@type'] == 'NewsArticle' and 'datePublished' in item:
                        date_published = item['datePublished']
                        break
        except Exception as e:
            print(f"Error parsing JSON-LD: {e}")

    # Parse the date if found
    if date_published != "No Date":
        try:
            date_published_dt = datetime.strptime(date_published, "%Y-%m-%dT%H:%M:%S%z")
            date_published = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
        except ValueError as e:
            print(f"Error parsing date: {e}")
    
    # Extract author
    author_tag = soup.find('span', class_='content_main_meta_author_item_name')
    author = author_tag.get_text(strip=True) if author_tag else "No Author"
    
    return {
        "Title": title,
        "Date Published": date_published,
        "Author": author,
        "URL": article_url,
        "Source": article_url.split('/')[2]
    }


In [114]:
def article_contents(article_url, storage_dir):
    
    try:
        # Загрузка HTML страницы
        response = requests.get(article_url)
        
        # Check for 404 errors or other HTTP status codes
        if response.status_code != 200:
            raise requests.HTTPError(f"HTTP Error: {response.status_code} for {article_url}")
        
        html_content = response.text
        
        # Парсинг HTML
        soup = BS(html_content, "html.parser")
        
        # Title extract
        try:
            title_tag = soup.find('h1')
            title = title_tag.get_text(strip=True) if title_tag else "No Title"
        except Exception: 
            title = "Title not found"
        
        # Author extract
        try:
            author_tag = soup.find('span', class_='content_main_meta_author_item_name')
            author = author_tag.get_text(strip=True) if author_tag else "No Author"
   
        except Exception: 
            author = "Author not found"
        # Extract date published
        json_ld_tags = soup.find_all('script', type="application/ld+json")
        date_published = "No Date"
        # Date extract
        for tag in json_ld_tags:
            try:
                # Parse the JSON content
                json_ld = json.loads(tag.string)
        
                # Check if it's a dictionary and has 'datePublished'
                if isinstance(json_ld, dict) and 'datePublished' in json_ld:
                    date_published = json_ld['datePublished']
                    break
                elif isinstance(json_ld, list):  # JSON-LD can also be a list of objects
                    for item in json_ld:
                        if '@type' in item and item['@type'] == 'NewsArticle' and 'datePublished' in item:
                            date_published = item['datePublished']
                            break
            except Exception as e:
                print(f"Error parsing JSON-LD: {e}")

        # Parse the date if found
        if date_published != "No Date":
            try:
                date_published_dt = datetime.strptime(date_published, "%Y-%m-%dT%H:%M:%S%z")
                date_published = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Error parsing date: {e}")
        # Content extract
        try:
            article_body = soup.find("div", class_="content_main_text")
            article_text = article_body.get_text(separator="\n", strip=True) if article_body else "Content not found"
        except Exception: 
            article_text = "Content not found"
        
        # Сохранение текста в файл
        valid_title = re.sub(r'[\\/:"*?<>|]+', '', title)  # Remove invalid characters
        filename = os.path.join(storage_dir, f"{valid_title}.txt")
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(f"{title}\n\n")
            file.write(article_text)
        
        print("Статья успешно сохранена!")
        
        return {
            "Title": title,
            "Date Published": date_published,
            "Author": author,
            "URL": article_url,
            "Source": article_url.split('/')[2],
            "Status": "Success"
        }
        
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [118]:
def main():
    
    key_word_input = input("Input key word:")
    base_url = "https://tengrinews.kz"
    search_url = base_url + "/search/?text="
    words = key_word_input.split()
    string = "+".join(words)
    key_word_url = search_url + string

    print(f"Ссылка по вашему запросу: \n{key_word_url}")
    
    # Creating directory with the name of the input 
    storage_dir = os.path.join(os.getcwd(), key_word_input)
    os.makedirs(storage_dir, exist_ok=True)

    response = requests.get(key_word_url) 
    html_content = response.text
    
    # Parsing HTML
    soup = BS(html_content, "html.parser")
    
    pages = 5 # Counting manually how much page usually indicates in the front page. 
    articles_count = 0
    urls_list = []

    print(f"Чило страниц:{pages}\n")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):  # Итерируемся от 1 до 5 включительно
        full_url = base_url + "/search/page/" + str(page) + "/?field=all&text=" + string +"&sort=date"
        print(f"Processing page:{page}\n")
        # Call the function and get the extracted URLs
        article_urls = extract_urls(base_url, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
        articles_count += len(article_urls)
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue
        
    print(f"Articles were found:{articles_count}")

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to text")

    articles_data = []  # To store article details for CSV

    for url in urls_list:
        print(url)
        article_detail = article_contents(url, storage_dir)
        articles_data.append(article_detail)

    # CSV file creation
    csv_file = os.path.join(storage_dir, f"{key_word_input}.csv")
    
    # CSV headers
    headers = ["Title", "Date Published", "Author", "URL", "Source", "Status"]
    
    # Save to CSV
    save_to_csv(articles_data, csv_file, headers)

if __name__ == "__main__":
    main()


Input key word:Донор
Ссылка по вашему запросу: 
https://tengrinews.kz/search/?text=Донор
Чило страниц:5

Processing page:1

https://tengrinews.kz/mixnews/mark-tsukerberg-meta-otkajetsya-faktchekinga-facebook-559053/

https://tengrinews.kz/usa/bayden-obratilsya-strane-svyazi-pobedoy-trampa-vyiborah-553322/

https://tengrinews.kz/kazakhstan_news/sledam-pavla-durova-kazahstantsyi-stanovyatsya-donorami-543721/

https://tengrinews.kz/curious/v-reanimatsii-povtoryal-imya-neznakomogo-cheloveka-istoriya-543236/

https://tengrinews.kz/world_news/vtoroy-patsient-vyilechilsya-ot-vich-v-germanii-541917/

https://tengrinews.kz/kazakhstan_news/si-tszinpin-napisal-statyu-obschem-stremlenii-kitaya-540060/

https://tengrinews.kz/kazakhstan_news/stat-geroem-prosto-sdat-stvolovyie-kletki-spasti-bolnogo-533819/

https://tengrinews.kz/kazakhstan_news/balanyin-fotosuretne-ruyina-karap-tandauga-boladyi-elmzde-531199/

https://tengrinews.kz/healthy/mogut-donoryi-krovi-poluchat-zakonnyie-otgulyi-kazahstane-507

Статья успешно сохранена!
https://tengrinews.kz/usa/bayden-obratilsya-strane-svyazi-pobedoy-trampa-vyiborah-553322/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/sledam-pavla-durova-kazahstantsyi-stanovyatsya-donorami-543721/
Статья успешно сохранена!
https://tengrinews.kz/curious/v-reanimatsii-povtoryal-imya-neznakomogo-cheloveka-istoriya-543236/
Статья успешно сохранена!
https://tengrinews.kz/world_news/vtoroy-patsient-vyilechilsya-ot-vich-v-germanii-541917/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/si-tszinpin-napisal-statyu-obschem-stremlenii-kitaya-540060/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/stat-geroem-prosto-sdat-stvolovyie-kletki-spasti-bolnogo-533819/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/balanyin-fotosuretne-ruyina-karap-tandauga-boladyi-elmzde-531199/
Статья успешно сохранена!
https://tengrinews.kz/healthy/mogut-donoryi-krovi-poluchat-zakonnyie-otgulyi-kazahstane-507042/
Статья успе

Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/izbejat-moshennichestva-okazanii-pomoschi-bolnyim-detyam-301732/
Статья успешно сохранена!
https://tengrinews.kz/usa/prestijnyiy-amerikanskiy-vuz-vyistupil-protiv-301127/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/dvuhletney-tahmine-abyishevoy-peresadili-kostnyiy-mozg-299141/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/kitay-vyinujden-postavlyat-svoih-rabochih-kazahstan-ekspert-288408/
Статья успешно сохранена!
https://tengrinews.kz/kazakhstan_news/pomenyat-operatora-svyazi-otmenyi-mobilnogo-rabstva-286640/
Статья успешно сохранена!
https://tengrinews.kz/medicine/v-kazahstane-vpervyie-nezakonno-izyyali-chelovecheskiy-organ-283261/
Статья успешно сохранена!
https://tengrinews.kz/story/kazahstanskiy-pevets-otvetil-post-razlichii-almatyi-astanyi-280092/
Статья успешно сохранена!
https://tengrinews.kz/europe/detyam-donorov-spermyi-razreshili-uznavat-imya-ottsa-lyubom-269204/
Статья успе

In [ ]:
### Azattyk too long name modification

In [8]:
def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('li', class_='col-xs-12 col-sm-12 col-md-12 col-lg-12 fui-grid__inner')
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]
    
    return full_urls

def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('li', class_='col-xs-12 col-sm-12 col-md-12 col-lg-12 fui-grid__inner')
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]
    
    return full_urls

def article_contents(article_url, storage_dir):
    try:
        # Загрузка HTML страницы
        response = requests.get(article_url)
        if response.status_code != 200:
            raise requests.HTTPError(f"HTTP Error: {response.status_code} for {article_url}")
        
        html_content = response.text
        soup = BS(html_content, "html.parser")
        
        # Title extract
        title_tag = soup.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "No Title"
        
        # Author extract
        author_tag = soup.find('a', class_='links__item-link')
        author = author_tag.get_text(strip=True) if author_tag else "No Author"
        
        # Extract date published
        date_tag = soup.find('time', pubdate="pubdate")
        if date_tag and date_tag.has_attr('datetime'):
            date_published = date_tag['datetime']
            date_published_dt = datetime.strptime(date_published, "%Y-%m-%dT%H:%M:%S%z")
            formatted_timestamp = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
        else:
            formatted_timestamp = "Date not found"
        
        # Content extract
        article_body = soup.find("div", class_="col-xs-12 col-sm-12 col-md-10 col-lg-10 pull-right")
        article_text = article_body.get_text(separator="\n", strip=True) if article_body else "Content not found"
        
        # Generate safe filename
        valid_title = re.sub(r'[\\/:"*?<>|]+', '', title)[:100]
        filename = os.path.join(storage_dir, f"{valid_title}.txt")
        
        # Save article to file
        with open(filename, "w", encoding="utf-8") as file:
            file.write(f"{valid_title}\n\n")
            file.write(article_text)
        
        print(f"Статья успешно сохранена как {filename}!")
        
        return {
            "Title": valid_title,
            "Date Published": formatted_timestamp,
            "Author": author,
            "URL": article_url,
            "Source": article_url.split('/')[2],
            "Status": "Success"
        }
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {
            "Title": "Error",
            "Date Published": "Error",
            "Author": "Error",
            "URL": article_url,
            "Source": article_url.split('/')[2],
            "Status": f"Error: {e}"
        }
    
def main():
    
    key_word_input = input("Input key word:")
    base_url = "https://www.azattyq.org/"
    search_url = base_url + "/s?k="
    words = key_word_input.split()
    string = "+".join(words)
    key_word_url = search_url + string

    print(f"Ссылка по вашему запросу: \n{key_word_url}")
    
    # Creating directory with the name of the input 
    storage_dir = os.path.join(os.getcwd(), key_word_input)
    os.makedirs(storage_dir, exist_ok=True)

    response = requests.get(key_word_url) 
    html_content = response.text
    
    # Parsing HTML
    soup = BS(html_content, "html.parser")
    
    article_founded = soup.find("span", class_ ="srch-result__results-count")
    articles = article_founded.get_text()

    num_article = re.findall(r'\d+', articles)
    num = int(num_article[0]) 
    
    # Each web-page in site only contains 5 articels, it can be also changed 
    articles_per_page = 10
    pages = math.ceil(num / articles_per_page) 

    print(f"Чило страниц:{pages}\n")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):  # Итерируемся от 1 до 5 включительно
        full_url = key_word_url + "&tab=any-content&pi=" + str(page) + "&r=any&pp=10"
        print(f"Processing page:{page}\n")
        # Call the function and get the extracted URLs
        article_urls = extract_urls(base_url, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue
        

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to text")

    articles_data = []  # To store article details for CSV

    for url in urls_list:
        print(url)
        article_detail = article_contents(url, storage_dir)
        articles_data.append(article_detail)

    # CSV file creation
    csv_file = os.path.join(storage_dir, f"{key_word_input}.csv")
    
    # CSV headers
    headers = ["Title", "Date Published", "Author", "URL", "Source", "Status"]
    
    # Save to CSV
    save_to_csv(articles_data, csv_file, headers)

if __name__ == "__main__":
    main()


Input key word:Донор
Ссылка по вашему запросу: 
https://www.azattyq.org//s?k=Донор
Чило страниц:16

Processing page:1

https://www.azattyq.org/a/kremlin-interference-2024-elections-eastern-europe-disinformation/33165336.html

https://www.azattyq.org/a/32814254.html

https://www.azattyq.org/a/32805928.html

https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.html

https://www.azattyq.org/a/32795027.html

https://www.azattyq.org/a/32706241.html

https://www.azattyq.org/a/kazakhsan-jusan-bank-central-asia-west-china/32504577.html

https://www.azattyq.org/a/32498757.html

https://www.azattyq.org/a/kazakhstan-media-review-tasmagambetov-putin-sanction/32384676.html

https://www.azattyq.org/a/32091469.html

Processing page:2

https://www.azattyq.org/a/32015084.html

https://www.azattyq.org/a/russia-ukraine-invasion-six-months-five-things-analysis/32002079.html

https://www.azattyq.org/a/31967325.html

https://www.azattyq.org/a/afghanistan-taliban-legitim

https://www.azattyq.org/a/1161400.html

https://www.azattyq.org/a/1156371.html

https://www.azattyq.org/a/1153502.html

https://www.azattyq.org/a/1150851.html

https://www.azattyq.org/a/1150829.html

https://www.azattyq.org/a/1149578.html

https://www.azattyq.org/a/1149513.html

https://www.azattyq.org/a/1149240.html

https://www.azattyq.org/a/1148599.html

https://www.azattyq.org/a/1148412.html

Processing page:16

https://www.azattyq.org/a/1148322.html

https://www.azattyq.org/a/1148308.html

https://www.azattyq.org/a/1147746.html

https://www.azattyq.org/a/1147108.html

https://www.azattyq.org/a/1146851.html

https://www.azattyq.org/a/1146669.html

https://www.azattyq.org/a/1146596.html


All Extracted URLs:

https://www.azattyq.org/a/kremlin-interference-2024-elections-eastern-europe-disinformation/33165336.html
https://www.azattyq.org/a/32814254.html
https://www.azattyq.org/a/32805928.html
https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.h

Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Кремль 2024 жылы сайлауларға араласқанда қолданатын бес амал.txt!
https://www.azattyq.org/a/32814254.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Израиль армиясы БҰҰ-ның Газадағы ғимараты астынан ХАМАС туннелін тапты.txt!
https://www.azattyq.org/a/32805928.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/HRW НеМолчи.kz қорының жетекшісін бейтарап тергеуге шақырды.txt!
https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Ресей журналистикасы өлген жоқ. Шетелге кеткен ресейлік журналистер қалай жұмыс істеп жатыр.txt!
https://www.azattyq.org/a/32795027.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Тоғыз мемлекет палестиналық босқындарға қаржылай көмекті тоқтатты.txt!
https://www.azattyq.org/a/32706241.html
Статья успешно сохр

Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Еуропа нарығына шығу – Қырғызстанға әлі арман.txt!
https://www.azattyq.org/a/27751313.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Стамбулда дүниежүзілік гуманитарлық саммит өтеді.txt!
https://www.azattyq.org/a/kazakhstan_environmental_organization_stopped_to_exist/27652615.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/«Каспий табиғаты» ұйымы қызметін тоқтатпақ.txt!
https://www.azattyq.org/a/kazakhstan_gongo_and_ngo/27568294.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/ҮЕҰ дербес деректерін ашуды міндеттейтін ережеге шағымданды.txt!
https://www.azattyq.org/a/27533357.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Донор елдер Сирияға 10 миллиард доллар көмек бермек.txt!
https://www.azattyq.org/a/27531510.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Лондонда Сирияға донор ел

Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/БҰҰ басшысы донор елдерді кедейлікпен күреске ақшаны аямауға шақырды.txt!
https://www.azattyq.org/a/2162727.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Әлем елдері Пәкістанға көбірек көмек бермекші.txt!
https://www.azattyq.org/a/2133820.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Пәкістанға 500 млн. долларға жуық қаржы жиналды.txt!
https://www.azattyq.org/a/2129760.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Дүниежүзілік банк Пәкістанға 900 млн. доллар ұсынды.txt!
https://www.azattyq.org/a/2111188.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Халықаралық донорлар Қырғызстанға 1,1 миллиард доллар бөледі.txt!
https://www.azattyq.org/a/Kyrgyzstan_International_donors_conference_in_Bishkek/2111167.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Шетелдік донорлар Қырғызстанға

Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Чернобыль апатының зардаптарын еңсеруге 170 млрд $ қажет.txt!
https://www.azattyq.org/a/1163884.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Ана болғысы келетін әйелдерге медицина көмегі шексіз, тек ниетіңіз бен қаржыңыз болса жеткілікті.txt!
https://www.azattyq.org/a/1163086.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Ауғанстандық ресми топ өкілдерінің Лондон конференциясының нәтижелеріне көңілі толып отыр.txt!
https://www.azattyq.org/a/1163050.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Халықаралық қауымдастық Ауғаныстанның алдағы 5 жылда елді қайта құруына 10,5 млрд доллар бөлеміз деп.txt!
https://www.azattyq.org/a/1163014.html
Статья успешно сохранена как /home/alikhan/Desktop/Data/Parsing/Донор/Еуроодақ министрлері Беларус, Ауғанстан, Иран және Ирақ мәселелерін талқылады.txt!
https://www.azattyq.org/a/1162952.html
Статья 

In [ ]:
### https://rus.azattyq.org/

In [2]:
def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('li', class_='col-xs-12 col-sm-12 col-md-12 col-lg-12 fui-grid__inner')
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]
    
    return full_urls

In [3]:
def save_to_csv(data_list, csv_file, headers):
    """Saves a list of article details to a CSV file."""
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data_list)
    print(f"CSV файл успешно сохранен в {csv_file}!")


In [6]:
def article_contents(article_url, storage_dir):
    
    try:
        # Загрузка HTML страницы
        response = requests.get(article_url)
        
        # Check for 404 errors or other HTTP status codes
        if response.status_code != 200:
            raise requests.HTTPError(f"HTTP Error: {response.status_code} for {article_url}")
        
        html_content = response.text
        
        # Парсинг HTML
        soup = BS(html_content, "html.parser")
        
        # Title extract
        try:
            title_tag = soup.find('h1')
            title = title_tag.get_text(strip=True) if title_tag else "No Title"
        except Exception: 
            title = "Title not found"
        
        
        # Author extract
        try:
            author_tag = soup.find('a', class_='links__item-link')
            author = author_tag.get_text(strip=True) if author_tag else "No Author"
   
        except Exception: 
            author = "Author not found"
        # Extract date published
        
        try:
            date_tag = soup.find('time',pubdate ="pubdate")
            if date_tag and date_tag.has_attr('datetime'):
                date_published = date_tag['datetime']
                date_published_dt = datetime.strptime(date_published, "%Y-%m-%dT%H:%M:%S%z")
                formatted_timestamp = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
            else:
                date_published = "Date not found"
        except Exception: 
            date_published = "Date not found"
    
        # Content extract
        try:
            article_body = soup.find("div", class_="col-xs-12 col-sm-12 col-md-10 col-lg-10 pull-right")
            article_text = article_body.get_text(separator="\n", strip=True) if article_body else "Content not found"
        except Exception: 
            article_text = "Content not found"
        
        # Сохранение текста в файл
        valid_title = re.sub(r'[\\/:"*?<>|]+', '', title)  # Remove invalid characters
        filename = os.path.join(storage_dir, f"{valid_title}.txt")
        max_title_length = 100  # Set max length for title
        valid_title = valid_title[:max_title_length]  # Truncate title if it's too long
        
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(f"{valid_title}\n\n")
            file.write(article_text)
        
        print("Статья успешно сохранена!")
        
        return {
            "Title": valid_title,
            "Date Published": formatted_timestamp,
            "Author": author,
            "URL": article_url,
            "Source": article_url.split('/')[2],
            "Status": "Success"
        }
        
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [7]:
def main():
    
    key_word_input = input("Input key word:")
    base_url = "https://rus.azattyq.org/"
    search_url = base_url + "/s?k="
    words = key_word_input.split()
    string = "+".join(words)
    key_word_url = search_url + string

    print(f"Ссылка по вашему запросу: \n{key_word_url}")
    
    # Creating directory with the name of the input 
    storage_dir = os.path.join(os.getcwd(), key_word_input)
    os.makedirs(storage_dir, exist_ok=True)

    response = requests.get(key_word_url) 
    html_content = response.text
    
    # Parsing HTML
    soup = BS(html_content, "html.parser")
    
    article_founded = soup.find("span", class_ ="srch-result__results-count")
    articles = article_founded.get_text()

    num_article = re.findall(r'\d+', articles)
    num = int(num_article[0]) 
    
    # Each web-page in site only contains 5 articels, it can be also changed 
    articles_per_page = 10
    pages = math.ceil(num / articles_per_page) 

    print(f"Чило страниц:{pages}\n")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):  # Итерируемся от 1 до 5 включительно
        full_url = key_word_url + "&tab=any-content&pi=" + str(page) + "&r=any&pp=10"
        print(f"Processing page:{page}\n")
        # Call the function and get the extracted URLs
        article_urls = extract_urls(base_url, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue
        

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to text")

    articles_data = []  # To store article details for CSV

    for url in urls_list:
        print(url)
        article_detail = article_contents(url, storage_dir)
        articles_data.append(article_detail)

    # CSV file creation
    csv_file = os.path.join(storage_dir, f"{key_word_input}.csv")
    
    # CSV headers
    headers = ["Title", "Date Published", "Author", "URL", "Source", "Status"]
    
    # Save to CSV
    save_to_csv(articles_data, csv_file, headers)

if __name__ == "__main__":
    main()


Input key word:Донор
Ссылка по вашему запросу: 
https://www.azattyq.org//s?k=Донор
Чило страниц:16

Processing page:1

https://www.azattyq.org/a/kremlin-interference-2024-elections-eastern-europe-disinformation/33165336.html

https://www.azattyq.org/a/32814254.html

https://www.azattyq.org/a/32805928.html

https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.html

https://www.azattyq.org/a/32795027.html

https://www.azattyq.org/a/32706241.html

https://www.azattyq.org/a/kazakhsan-jusan-bank-central-asia-west-china/32504577.html

https://www.azattyq.org/a/32498757.html

https://www.azattyq.org/a/kazakhstan-media-review-tasmagambetov-putin-sanction/32384676.html

https://www.azattyq.org/a/32091469.html

Processing page:2

https://www.azattyq.org/a/32015084.html

https://www.azattyq.org/a/russia-ukraine-invasion-six-months-five-things-analysis/32002079.html

https://www.azattyq.org/a/31967325.html

https://www.azattyq.org/a/afghanistan-taliban-legitim

https://www.azattyq.org/a/1161400.html

https://www.azattyq.org/a/1156371.html

https://www.azattyq.org/a/1153502.html

https://www.azattyq.org/a/1150851.html

https://www.azattyq.org/a/1150829.html

https://www.azattyq.org/a/1149578.html

https://www.azattyq.org/a/1149513.html

https://www.azattyq.org/a/1149240.html

https://www.azattyq.org/a/1148599.html

https://www.azattyq.org/a/1148412.html

Processing page:16

https://www.azattyq.org/a/1148322.html

https://www.azattyq.org/a/1148308.html

https://www.azattyq.org/a/1147746.html

https://www.azattyq.org/a/1147108.html

https://www.azattyq.org/a/1146851.html

https://www.azattyq.org/a/1146669.html

https://www.azattyq.org/a/1146596.html


All Extracted URLs:

https://www.azattyq.org/a/kremlin-interference-2024-elections-eastern-europe-disinformation/33165336.html
https://www.azattyq.org/a/32814254.html
https://www.azattyq.org/a/32805928.html
https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.h

Статья успешно сохранена!
https://www.azattyq.org/a/32814254.html
Статья успешно сохранена!
https://www.azattyq.org/a/32805928.html
Статья успешно сохранена!
https://www.azattyq.org/a/russia-independent-media-abroad-emigre-journalism-alive/32800811.html
Статья успешно сохранена!
https://www.azattyq.org/a/32795027.html
Статья успешно сохранена!
https://www.azattyq.org/a/32706241.html
Статья успешно сохранена!
https://www.azattyq.org/a/kazakhsan-jusan-bank-central-asia-west-china/32504577.html
Статья успешно сохранена!
https://www.azattyq.org/a/32498757.html
Статья успешно сохранена!
https://www.azattyq.org/a/kazakhstan-media-review-tasmagambetov-putin-sanction/32384676.html
Статья успешно сохранена!
https://www.azattyq.org/a/32091469.html
Статья успешно сохранена!
https://www.azattyq.org/a/32015084.html
Статья успешно сохранена!
https://www.azattyq.org/a/russia-ukraine-invasion-six-months-five-things-analysis/32002079.html
Статья успешно сохранена!
https://www.azattyq.org/a/31967325.htm

Статья успешно сохранена!
https://www.azattyq.org/a/Kabul_conference_Karzai_urges_Afghan_forces_be_ready_to_protect_the_country_2014/2104974.html
Статья успешно сохранена!
https://www.azattyq.org/a/UN_peacekeepers_mission_and_the_Osh_events/2095201.html
Статья успешно сохранена!
https://www.azattyq.org/a/Osh_events_damage_calculated/2097958.html
Статья успешно сохранена!
https://www.azattyq.org/a/Kazakhstan_press_digest_china_credit/1997537.html
Статья успешно сохранена!
https://www.azattyq.org/a/1924764.html
Статья успешно сохранена!
https://www.azattyq.org/a/1924055.html
Статья успешно сохранена!
https://www.azattyq.org/a/afghanistan_government_corruption_president_parliament/1920122.html
Статья успешно сохранена!
https://www.azattyq.org/a/1872985.html
Статья успешно сохранена!
https://www.azattyq.org/a/1829065.html
Статья успешно сохранена!
https://www.azattyq.org/a/1815770.html
Статья успешно сохранена!
https://www.azattyq.org/a/1770943.html
Статья успешно сохранена!
https://www.az

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
### Informburo https://informburo.kz/new-search?query=Донор

In [56]:
def extract_urls(base_url,search_url):
    response = requests.get(search_url) # It is the default
    html_content = response.text
    
    # Парсинг HTML
    soup = BS(html_content, "html.parser")
    
    divs = soup.find_all('div', class_='uk-width-1-3@m')

    # Extract URLs from <a> tags inside those divs
    urls = [div.a['href'] for div in divs if div.a]
    full_urls = [base_url.rstrip('/') + div.a['href'] for div in divs if div.a]
    
    return full_urls

In [57]:
def save_to_csv(data_list, csv_file, headers):
    """Saves a list of article details to a CSV file."""
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data_list)
    print(f"CSV файл успешно сохранен в {csv_file}!")

In [58]:
def article_contents(article_url, storage_dir):
    
    try:
        # Загрузка HTML страницы
        response = requests.get(article_url)
        
        # Check for 404 errors or other HTTP status codes
        if response.status_code != 200:
            raise requests.HTTPError(f"HTTP Error: {response.status_code} for {article_url}")
        
        html_content = response.text
        
        # Парсинг HTML
        soup = BS(html_content, "html.parser")
        
        # Title extract
        try:
            title_tag = soup.find('h1')
            title = title_tag.get_text(strip=True) if title_tag else "No Title"
        except Exception: 
            title = "Title not found"
        
        
        # Author extract
        try:
            author_tag = soup.find("small", class_= "article-meta-authors uk-text-muted")
            author = author_tag.get_text(strip=True) if author_tag else "No Author"
   
        except Exception: 
            author = "Author not found"
        # Extract date published
        
        try:
            date_tag = soup.find('time')
            if date_tag and date_tag.has_attr('datetime'):
                date_published = date_tag['datetime']
                date_published_dt = datetime.strptime(date_published, "%Y-%m-%dT%H:%M:%S%z")
                formatted_timestamp = date_published_dt.strftime("%Y-%m-%d %H:%M:%S")
            else:
                date_published = "Date not found"
        except Exception: 
            date_published = "Date not found"
    
        # Content extract
        try:
            article_body = soup.find("div", class_="uk-width-2-3@m uk-width-1-1")
            article_text = article_body.get_text(separator="\n", strip=True) if article_body else "Content not found"
        except Exception: 
            article_text = "Content not found"
        
        # Сохранение текста в файл
        valid_title = re.sub(r'[\\/:"*?<>|]+', '', title)  # Remove invalid characters
        filename = os.path.join(storage_dir, f"{valid_title}.txt")
        max_title_length = 100  # Set max length for title
        valid_title = valid_title[:max_title_length]  # Truncate title if it's too long
        
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(f"{valid_title}\n\n")
            file.write(article_text)
        
        print("Статья успешно сохранена!")
        
        return {
            "Title": valid_title,
            "Date Published": formatted_timestamp,
            "Author": author,
            "URL": article_url,
            "Source": article_url.split('/')[2],
            "Status": "Success"
        }
        
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [60]:
def main():
    
    key_word_input = input("Input key word:")
    base_url = "https://informburo.kz"
    search_url = base_url + "/new-search?query="
    words = key_word_input.split()
    string = "+".join(words)
    key_word_url = search_url + string

    print(f"Ссылка по вашему запросу: \n{key_word_url}")
    
    # Creating directory with the name of the input 
    storage_dir = os.path.join(os.getcwd(), key_word_input)
    os.makedirs(storage_dir, exist_ok=True)

    pages = 57 # Counting manually how much page usually indicates in the front page. 
    articles_count = 0
    urls_list = []

    print(f"Чило страниц:{pages}\n")
    
    urls_list = []  # Initialize the list

    for page in range(1, pages + 1):  # Итерируемся от 1 до 5 включительно
        full_url = key_word_url + "&page=" + str(page)
        print(full_url)
        print(f"Processing page:{page}\n")
        article_urls = extract_urls(base_url, full_url)
    
        for article_url in article_urls: 
            print(f"{article_url}\n")    
    
        # Append URLs to the list
        urls_list.extend(article_urls) 
    
    
        # Skip if no URLs are found
        if not article_urls:  
            print(f"Skipping page {page} due to no articles found.\n")
        continue
        

    # The final list of URLs
    print("\nAll Extracted URLs:\n")

    for url in urls_list:
        print(url)
    
    print(f"\nCount:{len(urls_list)}\n")
    print("Starting saving all the extracted articles to text")

    articles_data = []  # To store article details for CSV

    for url in urls_list:
        print(url)
        article_detail = article_contents(url, storage_dir)
        articles_data.append(article_detail)

    # CSV file creation
    csv_file = os.path.join(storage_dir, f"{key_word_input}.csv")
    
    # CSV headers
    headers = ["Title", "Date Published", "Author", "URL", "Source", "Status"]
    
    # Save to CSV
    save_to_csv(articles_data, csv_file, headers)

if __name__ == "__main__":
    main()

Input key word:Донор
Ссылка по вашему запросу: 
https://informburo.kz/new-search?query=Донор
Чило страниц:57

https://informburo.kz/new-search?query=Донор&page=1
Processing page:1

https://informburo.kz/fotoreportazh/zacem-kazaxstancy-sdayut-svoyu-krov-i-skolko-polucayut-donory

https://informburo.kz/stati/transplantaciya-kostnogo-mozga-kak-eta-bezopasnaya-dlya-donora-procedura-mozet-spasti-sotni-pacientov-s-rakom-krovi

https://informburo.kz/stati/skolko-donorov-v-kazaxstane-i-skolko-ziznei-oni-spasli

https://informburo.kz/special/put-donora-bezvozmezdnyx-donacii-krovi-v-astane-stanovitsya-vsyo-bolse

https://informburo.kz/kaz/newskaz/qazaqstanda-qurmetti-donor-tosbelgisi-qaita-tabystalady

https://informburo.kz/stati/put-krovi-otkuda-berutsya-donory-i-kak-preparaty-dlya-perelivaniya-popadayut-v-bolnicy

https://informburo.kz/stati/trebuyutsya-donory-7248.html

https://informburo.kz/novosti/mangistauskaya-oblast-poteryaet-status-regiona-donora-respublikanskogo-byudzheta-v-2023-godu



https://informburo.kz/novosti/rodnye-donora-iz-aktobe-u-kotorogo-izyali-pochki-trebuyut-vyzvat-v-sud-religioveda.html

https://informburo.kz/novosti/arestovannogo-kostanayca-povtorno-etapiruyut-v-astanu-dlya-spaseniya-zhizni-onkobolnoy-sestry.html

https://informburo.kz/novosti/organy-52-letney-zhitelnicy-pavlodara-peresadili-chetverym-kazahstancam.html

https://informburo.kz/novosti/v-vko-ne-hvataet-donorskoy-krovi-6476.html

https://informburo.kz/novosti/kazhdyy-umirayushchiy-ot-travm-dolzhen-soglasitsya-stat-donorom-chtoby-pokryt-deficit-organov.html

https://informburo.kz/novosti/v-avstrii-vypustili-zhurnal-s-krovyu-vich-inficirovannyh-na-stranicah-4025.html

https://informburo.kz/novosti/sotrudniki-conov-strany-sdali-280-litrov-krovi.html

https://informburo.kz/novosti/v-nur-sultane-otkryli-novyj-art-obuekt

https://informburo.kz/novosti/nalogoviki-vernuli-okolo-100-mlrd-tenge-v-byudzhet-posle-proverok-nedropolzovatelej

https://informburo.kz/kaz/omir/uryq-donory-zasandy-tusik-zan

https://informburo.kz/novosti/v-almaty-proveli-unikalnuyu-dlya-kazahstana-operaciyu-po-peresadke-pecheni.html

https://informburo.kz/stati/v-aktau-obezvrezhena-upravlyaemaya-termoyadernaya-bomba-no-eyo-othody-budut-opasny-eshchyo-50-let-13483.html

https://informburo.kz/novosti/peresadku-kostnogo-mozga-tahmine-abyshevoy-ne-sdelali-v-kazahstane-iz-za-devalvacii.html

https://informburo.kz/novosti/pochki-ot-umershego-cheloveka-vpervye-peresadili-bolnym-v-aktobe.html

https://informburo.kz/novosti/dorogie-patsienty-iz-kazahstana-957.html

https://informburo.kz/novosti/mamin-iz-za-pandemii-kolichestvo-operacij-po-transplantacii-organov-sokratilos-v-dva-raza

https://informburo.kz/stati/iscelilsya-muzcina-40-let-bolevsii-vic-eto-vsego-cetvyortyi-podobnyi-slucai

https://informburo.kz/novosti/den-otkrytyh-dverej-po-voprosam-transplantacii-pecheni-i-pochek-projdyot-v-astane

https://informburo.kz/novosti/ne-budut-lecit-ctoby-bystree-zabrat-organy-pocemu-kazaxstancy-ne-resayutsya-na-posmertnoe

https://informburo.kz/stati/mladenec-kotoromu-27-let-v-ssha-poyavilas-na-svet-devochka-zhdavshaya-rozhdeniya-dolshe-vseh-v-mire.html

https://informburo.kz/novosti/oon-prosit-bolee-400-mln-dollarov-dlya-pomoshchi-nepalu-3719.html

https://informburo.kz/stati/my-pobedili-vmeste-kak-prohodyat-igry-chempionov-v-nur-sultane.html

https://informburo.kz/novosti/mne-rk-predstavilo-zakonoproekt-o-povyshenii-nalogovyh-stavok-v-shymkente-i-turkestane.html

https://informburo.kz/novosti/sportivnuyu-shkolu-za-50-mln-tenge-otkryli-na-okraine-astany.html

https://informburo.kz/stati/npo-v-kazahstane-kak-gosudarstvo-vzaimodeystvuet-s-obshchestvennikami-.html

https://informburo.kz/novosti/vrachi-doveli-studentku-iz-astany-do-sostoyaniya-komy-i-brosili-na-proizvol-sudby-mat-bolnoy.html

https://informburo.kz/novosti/mat-devochki-stradayushchey-leykozom-prosit-nazarbaeva-pomoch-v-poluchenii-kvoty-dlya-docheri-27935.html

https://informburo.kz/cards/chem-zanimaetsya-nedavno-sozdannoe-ministerstvo-po-del

https://informburo.kz/kaz/newskaz/qazaqstan-onirleri-respublikalyq-byudzetten-qansa-aqsa-alady-zane-qansa-beredi-infografika

https://informburo.kz/kaz/kogam/kolsai-kolderinin-direktory-ulttyq-sayabaqqa-kelusilerdi-sekteu-tazalyqty-saqtauga-komektesedi

https://informburo.kz/stati/lyudi-kotorye-ne-ispugalis-vospominaniya-o-tragiceskom-yanvare

https://informburo.kz/kaz/newskaz/syucai-numerologiyanyn-bir-turi-qmdb-ukimi-qandai

https://informburo.kz/novosti/zaklyucyonnym-v-ssa-predlozili-sokratit-sroki-v-obmen-na-donorstvo-organov

https://informburo.kz/novosti/v-riddere-ne-budut-povysat-tarify-na-teplo-posle-stroitelstva-zavoda-donora-dlya-tec

https://informburo.kz/novosti/cto-proisxodit-v-ukraine-glavnoe-za-den-4-fevralya

https://informburo.kz/novosti/zitelyam-riddera-vernut-63-mln-tenge-za-nedopostavlennoe-teplo

https://informburo.kz/novosti/na-tec-v-riddere-dalo-sboi-novoe-oborudovanie

https://informburo.kz/stati/golod-razruxa-xolera-a-teper-zemletryasenie-pocemu-siriya-postrada

...

Статья успешно сохранена!
https://informburo.kz/stati/ucyonym-udalos-sozdat-celoveceskii-embrion-iz-odnoi-stvolovoi-kletki-kak-eto-vozmozno-a-glavnoe-zacem
Статья успешно сохранена!
https://informburo.kz/novosti/cto-proicxodit-v-ukraine-glavnoe-k-utru-21-iyunya
Статья успешно сохранена!
https://informburo.kz/novosti/tokaev-na-forume-volontyorov-miloserdie-kacestvo-prisushhee-kazaxskomu-narodu
Статья успешно сохранена!
https://informburo.kz/kaz/tusyndirme/qazaqstandyqtar-paterinin-texnikalyq-pasportyn-auystyruy-tiis-ol-usin-ne-qazet
Статья успешно сохранена!
https://informburo.kz/novosti/spartakiada-dlya-detei-kotorye-naxodyatsya-v-remissii-posle-onkologiceskix-zabolevanii-prosla-v-astane
Статья успешно сохранена!
https://informburo.kz/novosti/cto-proicxodit-v-ukraine-glavnoe-k-utru-26-iyulya
Статья успешно сохранена!
https://informburo.kz/novosti/komissiya-ustanovila-veroyatnyi-istocnik-inficirovaniya-vic-v-cgkb-almaty
Статья успешно сохранена!
https://informburo.kz/kaz/tusyndirme/tung

Статья успешно сохранена!
https://informburo.kz/kaz/balalar-allergiyasy-nege-ayrysha-aupt-drger-kees.html
Статья успешно сохранена!
https://informburo.kz/kaz/it-etnen-as-zrleytn-dmhanalardy-zhabudy-synbapyz-toaeva-hat-zhazan-brizhit-bardo-oryny-klmen-shbat.html
Статья успешно сохранена!
https://informburo.kz/kaz/azastan-europada-balalar-suicidnen-alashy-zhet-eld-atarynda-zhaalytara-sholu.html
Статья успешно сохранена!
https://informburo.kz/pikir/kainar-oljay/zhapon-men-aza-zhayn-men-ashy.html
Статья успешно сохранена!
https://informburo.kz/stati/apostrofy-ili-digrafy-diskussiya-o-latinice-prodolzhaetsya.html
Статья успешно сохранена!
https://informburo.kz/pikir/kainar-oljay/ambash.html
Статья успешно сохранена!
https://informburo.kz/stati/chtoby-stat-policeyskim-pridyotsya-snachala-proyti-test-u-borcov-s-korrupciey.html
Статья успешно сохранена!
https://informburo.kz/novosti/samolyot-aviakompanii-bek-air-poterpel-krushenie-v-v-almatinskoy-oblasti.html
Статья успешно сохранена!
https://

Статья успешно сохранена!
https://informburo.kz/stati/koronakrizis-budet-samym-razoritelnym-za-stoletie-pochemu-ssha-postradayut-menshe-drugih.html
Статья успешно сохранена!
https://informburo.kz/novosti/po-faktu-chp-s-obvalom-na-rudnike-akmolinskoy-oblasti-nachali-proverku.html
Статья успешно сохранена!
https://informburo.kz/stati/ya-ne-hochu-detey-monologi-kazahstanskih-chayldfri.html
Статья успешно сохранена!
https://informburo.kz/novosti/parlamentskie-vybory-v-kyrgyzstane-naznachili-na-20-dekabrya-2020-goda-112923.html
Статья успешно сохранена!
https://informburo.kz/stati/byudzhet-almaty-v-2020-godu-vyrastet-na-37-iz-chego-formiruyutsya-dohody-goroda-i-na-chto-ih-potratyat.html
Статья успешно сохранена!
https://informburo.kz/novosti/kazahstan-vtroe-uvelichil-rashody-na-borbu-s-bednostyu.html
Статья успешно сохранена!
https://informburo.kz/novosti/nazarbaev-ekonomika-almaty-v-pyat-raz-prevyshaet-vvp-kyrgyzstana-i-tadzhikistana.html
Статья успешно сохранена!
https://informburo.kz/car

Статья успешно сохранена!
https://informburo.kz/stati/otcy-i-deti-parlamentskie-partii-razglyadeli-v-strane-obostrenie-konflikta-pokoleniy.html
Статья успешно сохранена!
https://informburo.kz/stati/kazashka-zhdyot-v-irake-svoego-spasitelya-9151.html
Статья успешно сохранена!
https://informburo.kz/novosti/kogda-nachnyotsya-stroitelstvo-bakad-rasskazal-zhenis-kasymbek.html
Статья успешно сохранена!
https://informburo.kz/novosti/v-kazahstane-naschitali-okolo-polumilliona-bezrabotnyh-.html
Статья успешно сохранена!
https://informburo.kz/novosti/v-astane-kardiohirurgi-vpervye-v-mire-implantirovali-besprovodnuyu-sistemu-dlya-podderzhki-serdca.html
Статья успешно сохранена!
https://informburo.kz/novosti/dariga-nazarbaeva-ya-za-to-chtoby-upravleniya-obrazovaniya-v-kazahstane-zakryli.html
Статья успешно сохранена!
https://informburo.kz/special/nurlan-nogaev-est-chyotkoe-videnie-togo-chto-predstoit-sdelat-v-atyrauskoy-oblasti.html
Статья успешно сохранена!
https://informburo.kz/novosti/nazarbaev

Статья успешно сохранена!
https://informburo.kz/novosti/bolnicy-vmesto-restoranov-i-barov-budet-stroit-baybek-v-almaty.html
Статья успешно сохранена!
https://informburo.kz/novosti/v-ssha-snova-zayavlyayut-o-zhelanii-vyjti-iz-voz
Статья успешно сохранена!
https://informburo.kz/kaz/newskaz/usaq-apatynan-aman-qalgandar-usin-115-adam-qan-tapsyrdy
Статья успешно сохранена!
https://informburo.kz/novosti/kostanaycy-sdayut-krov-dlya-pyatiletney-devochki-kotoraya-mesyac-lezhit-v-kome.html
Статья успешно сохранена!
CSV файл успешно сохранен в /home/alikhan/Desktop/Data/Parsing/Донор/Донор.csv!


In [ ]:
### KZ Pravda https://kazpravda.kz/search/?q=%D0%94%D0%BE%D0%BD%D0%BE%D1%80

In [74]:
response = requests.get("https://kazpravda.kz/search/?q=%D0%94%D0%BE%D0%BD%D0%BE%D1%80")
        
html_content = response.text
        
# Парсинг HTML
soup = BS(html_content, "html.parser")

In [75]:
print(html_content)




<!doctype html>
<html prefix="og: https://ogp.me/ns#">
<head>
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <title>
    Поиск
</title>
    
    <link href="https://kazpravda.kz/search/" rel="canonical">

    
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="csrfmiddlewaretoken" content="gDzgYKjAQZf6wy95QBpCF5F3pPytVsvCW7k1HS7YW8ZA2vJDDEJJmFtBd72DUAHA">
    
    <meta name="format-detection" content="telephone=no">
    <!-- This make sence for mobile browsers. It means, that content has been optimized for mobile browsers -->
    <meta name="HandheldFriendly" content="true">
    <!-- Stylesheet -->
    <link rel="shortcut icon" href="/static/web/img/favicon.ico"/>
     
    <link href="/static/web/css/main.css?v=90" rel="stylesheet" type="text/css">
    <link href="/static/web/css/separate-css/custom.css" rel="stylesheet" type="text/css">
    <link href="/static/web/css/backend.css?v=13" rel="stylesheet